## Introduction
Filter-blocks repository contains a collection of hardware digital filter implementations coded with myhdl.


### object oriented API
An API has been built for the filter-blocks package which allows easy access to the filter implementations. It also helps set parameters required by serving as a wrapper around MyHDL methods. 

### filter implementations 

The filter implementations that presently exist in the filter blocks package:

- fir filters
    -  direct form I 
    -  cascaded second order sections 
- iir filters
    -  direct form I
    -  cascaded second order sections
    -  parallel implementation


### Create FIR filter with integer coefficients
Here is an example of how you can create a filter with integer coefficients. In this example, scipy library is used to generate floating point filter coefficients which are then converted to integers.
#### Create filter coefficients

```python 
import scipy.signal as signal

fs = 1000.
f1 = 45.
f2 = 95.
b = signal.firwin(3,[f1/fs*2,f2/fs*2])    #3 taps
```
Next, we need to convert the coefficients to integers for which they will need to be scaled. For this example we assume, the coefficients are scaled to 18 bits. The exponent of the scale factor is determined as follows:

```python
B1 = 18 # Number of bits for coefficients
L1 = math.floor(math.log((2**(B1-1)-1)/max(b), 2))  #Scale factor exponent
bsc = b*(2**L1) #scaled coefficients
bsc_int = [int(x) for x in bsc] #convert coefficient to type int
```
#### Build the FIR Filter
``` python 
from filter_blocks.fda import FilterFIR
hdlfilter = FilterFIR()
```
#### Set the Filter Parameters to Work with Integers
Next, we set the input parameters of the filter to appropriate values.

```python 
hdlfilter.set_word_format(coeff_w = (18, 17, 0), input_w = (12,11,0), output_w = (24,23,0))
```
Args:<br>
coeff_w (tuple of int): word format (W,WI,WF)<br>
input_w (tuple of int): word format (W,WI,WF)<br>
output_w (tuple of int): word format (W,WI,WF)<br>
Where WI is the number of integer bits, WF is the number of fractional bit and W = WI+WF+1 (extra 1 for the sign bit). It is important to set the coefficient bits to 18, as this was used to calculate the scaling factor. In this example we are assuming that the input signal is of 12 bits and the output is set to 24 bits.  

Next, we pass the filter coefficients as determined earlier.
```python
hdlfilter.set_coefficients(coeff_b = bsc_int)

```


#### Create a Stimulus for the Filter
We generate an input signal (stimulus) for the filter by following the same steps that were used for the coefficients, discussed previously. In this example, create a signal which is a sine wave:

```python 
import numpy as np

N=20 #length of input signal
sig = [np.sin(0.1*np.pi*i) for i in np.arange(0,N,1)]

B2 = 12 # Number of bits
L2 = math.floor(math.log((2**(B2-1)-1)/max(sig), 2))  # calculate scaling factor

sig = np.multiply(sig, 2**L2) 
sig = sig.round()
sig = sig.astype(int)

hdlfilter.set_stimulus(sig) #pass the stimulus to the filter object

```

#### Run the simulation

To filter the input signal generated above, we need to run a simulation and obtain the output signal. Enter the following:

```python 
hdlfilter.run_sim()
y = hdlfilter.get_response()

```
Filter-blocks returns a numpy array of the output which is stored in 'y'. Here y is a 24 bits as was set earlier, meaning that some bits might have possibly been discarded to fit the output. The most significant 24 bits from the accumulator are assigned to y.


#### Converting design to Verilog or VHDL
```python 
hdlfilter.convert(hdl = 'VHDL')
```

hdl: 'VHDL' or 'Verilog'. Defaults to Verilog.

Args:<br>
path: Destination folder. Defaults to current working dir.<br>
name: Module and output file name. Defaults to self.mod.name.<br>
trace: Whether the testbench should dump all signal waveforms. Defaults to False.<br>
testbench: Verilog only. Specifies whether a testbench should be created. Defaults to True.<br>
timescale: timescale parameter. Defaults to '1ns/10ps'. Verilog only.<br>

### Create IIR filter with integer coefficients

The steps to be followed are the same as that of FIR filter. Hence only an example should suffice with minimal explanation. 

#### Create filter coefficients

In this example we will be using the scipy library to create iir coefficients for a low pass elliptical filter. We design a 3rd-order lowpass elliptic filter with 5 dB of passband ripple, 40 dB of stopband attenuation, and a passband edge frequency of 0.6$\pi$  rad/sample

```python 
import scipy.signal as signal

b, a = signal.ellip(3, 5, 40, 0.6, output='ba') # 3 taps 
```
Next, the coefficients are scaled up.

```python
B1 = 18 # Number of bits for coefficients
bsc = b*(2**B1) #scaled coefficients
asc = a*(2**B1) 
bsc_int = [int(x) for x in bsc] #convert coefficient to type int
asc_int = [int(x) for x in bsc] 
```
#### Build the IIR Filter
``` python 
from filter_blocks.fda import FilterIIR
hdlfilter = FilterIIR()
```
#### Set the Filter Parameters to Work with Integers
Next, we set the input parameters of the filter to appropriate values.

```python 
hdlfilter.set_word_format(coeff_w = (18, 17, 0), input_w = (12,11,0), output_w = (24,23,0))
```
Args:<br>
coeff_w (tuple of int): word format (W,WI,WF)<br>
input_w (tuple of int): word format (W,WI,WF)<br>
output_w (tuple of int): word format (W,WI,WF)<br>
Where WI is the number of integer bits, WF is the number of fractional bit and W = WI+WF+1 (extra 1 for the sign bit). It is important to set the coefficient bits to 18, as this was used to calculate the scaling factor. In this example we are assuming that the input signal is of 12 bits and the output is set to 24 bits.  

Next, we pass the filter coefficients as determined earlier.
```python
hdlfilter.set_coefficients(coeff_b = bsc_int, coeff_a = asc_int)

```

the info method prints details about the filter implementation 

```python 
hdlfilter.info()
```


#### Create a Stimulus for the Filter
We generate an input signal (stimulus) exactly the same way as for the fir filter. 

```python 
import numpy as np

N=20 #length of input signal
sig = [np.sin(0.1*np.pi*i) for i in np.arange(0,N,1)]

B2 = 12 # Number of bits
L2 = math.floor(math.log((2**(B2-1)-1)/max(sig), 2))  # calculate scaling factor

sig = np.multiply(sig, 2**L2) 
sig = sig.round()
sig = sig.astype(int)

hdlfilter.set_stimulus(sig) #pass the stimulus to the filter object

```

#### Run the simulation

To filter the input signal generated above, we need to run a simulation and obtain the output signal. Enter the following:

```python 
hdlfilter.run_sim()
y = hdlfilter.get_response()
```
Filter-blocks returns a numpy array of the output which is stored in 'y'. Here y is a 24 bits as was set earlier, meaning that some bits might have possibly been discarded to fit the output. The most significant 24 bits from the accumulator are assigned to y.


#### Converting design to Verilog or VHDL
```python 
hdlfilter.convert(hdl = 'VHDL')
```
hdl: 'VHDL' or 'Verilog'. Defaults to Verilog.

Args:<br>
path: Destination folder. Defaults to current working dir.<br>
name: Module and output file name. Defaults to self.mod.name.<br>
trace: Whether the testbench should dump all signal waveforms. Defaults to False.<br>
testbench: Verilog only. Specifies whether a testbench should be created. Defaults to True.<br>
timescale: timescale parameter. Defaults to '1ns/10ps'. Verilog only.<br>